In [4]:
import json
from tqdm import tqdm
import numpy as np

In [4]:
dict_p = {}
dict_d = {}
with open('./total_detail_new.json') as f:
    for i in tqdm(f):
        app = json.loads(i)
        game_info = app['game_info']
        publisher = game_info['publisher']
        developer = game_info['developer']
        review_count = game_info['review_count']
        for pber in publisher:
            if pber in dict_p.keys():
                dict_p[pber].append(review_count)
            else:
                dict_p[pber] = [review_count]
        for depr in developer:
            if depr in dict_d.keys():
                dict_d[depr].append(review_count)
            else:
                dict_d[depr] = [review_count]

18710it [02:00, 155.53it/s]


In [5]:
publisher = {}
with open('./publisher.json','r') as f:
    for i in f:
        pub = json.loads(i)
        publisher[pub['name']] = (pub['steam_followers'], pub['tweet_followers'],(pub['steam_followers'] + pub['tweet_followers'] if pub['tweet_followers'] else pub['steam_followers'])/2)

In [57]:
def longer_common_sequence(str1, str2):
    """
    最大公共子序列 实现
    """
    str1 = str1.lower()
    str2 = str2.lower()
    len1 = len(str1)
    len2 = len(str2)
    max_lcq_len = 0
    max_len_axis = (0, 0)
    lcq_matrix = [[0 for j in range(len2+1)] for i in range(len1+1)]
    for i, char_1 in enumerate(str1):
        for j, char_2 in enumerate(str2):
            if char_1 == char_2:
                lcq_matrix[i+1][j+1] = lcq_matrix[i][j] + 1
                if lcq_matrix[i+1][j+1] > max_lcq_len:
                    max_lcq_len = lcq_matrix[i+1][j+1]
                    max_len_axis = (i, j)
            else:
                lcq_matrix[i+1][j+1] = max(lcq_matrix[i+1][j], lcq_matrix[i][j+1])
    return max_lcq_len

In [33]:
def find_lcsubstr(s1, s2): 
    s1 = s1.lower()
    s2 = s2.lower()
    m = [[0 for i in range(len(s2)+1)]  for j in range(len(s1)+1)]
    mmax=0   #最长匹配的长度
    p=0  #最长匹配对应在s1中的最后一位
    for i in range(len(s1)):
        for j in range(len(s2)):
            if s1[i]==s2[j]:
                m[i+1][j+1]=m[i][j]+1
                if m[i+1][j+1]>mmax:
                    mmax=m[i+1][j+1]
                    p=i+1
    return mmax

# publisher

In [275]:
result_p = {}
for k,v in dict_p.items():
    if len(v) >= 10:
        result_p[k] = {'game_review':(int(np.mean(v)),int(np.sum(v)),len(v),np.max(v),np.max(v)/np.sum(v)),'followers':None}

In [276]:
publisher_key = list(publisher.keys())
for k,v in tqdm(result_p.items()):
    if k in publisher.keys():
        result_p[k]['match_name'] = k
        result_p[k]['followers'] = publisher[k]
    else:
        max_index = np.argmax([longer_common_sequence(k,x) for x in publisher_key])
        match_name = publisher_key[max_index]
        result_p[k]['followers'] = publisher[match_name]
        result_p[k]['match_name'] = match_name

100%|██████████| 257/257 [01:18<00:00,  3.26it/s]


In [277]:
output = []
for k,v in result_p.items():
    output.append({'name':k,'game_review':v['game_review'],'followers':v['followers'],'match_name':v['match_name']})

In [279]:
output = sorted(output,key=lambda x:x['followers'][2],reverse=True)

In [286]:
p_result = [x for x in output if x['followers'][0] > 100000 or (x['followers'][1] and x['followers'][1] > 100000)]

In [7]:
result_p = {}
for k,v in dict_p.items():
    result_p[k] = {'game_review':(int(np.mean(v)),int(np.sum(v)),len(v),np.max(v),np.max(v)/np.sum(v)),'followers':None}
famous_org = [x for x in publisher_key if publisher[x][0] > 100000 or (publisher[x][1] and publisher[x][1] > 100000)]
publisher_key = list(publisher.keys())
publisher_key_nomatch = [x for x in publisher_key if x not in result_p.keys()]
print(len(publisher_key_nomatch))
for k,v in tqdm(result_p.items()):
    if k in publisher.keys():
        result_p[k]['match_name'] = k
        result_p[k]['followers'] = publisher[k]
    elif result_p[k]['game_review'][0] >= 1000:
        max_index = np.argmax([longer_common_sequence(k,x) for x in publisher_key_nomatch])
        match_name = publisher_key_nomatch[max_index]
        result_p[k]['followers'] = publisher[match_name]
        result_p[k]['match_name'] = match_name
    else:
        result_p[k]['followers'] = None
        result_p[k]['match_name'] = None

5398


100%|██████████| 8178/8178 [10:29<00:00, 12.99it/s] 


In [8]:
output = []
for k,v in result_p.items():
    if v['followers']:
        output.append({'name':k,'game_review':v['game_review'],'followers':v['followers'],'match_name':v['match_name']})

In [9]:
p_result = [x for x in output if x['followers'][0] > 100000 or (x['followers'][1] and x['followers'][1] > 100000)]

In [38]:
result_p = {}
for k,v in dict_p.items():
    result_p[k] = {'game_review':(int(np.mean(v)),int(np.sum(v)),len(v),np.max(v),np.max(v)/np.sum(v)),'followers':None}

    
result = []
famous_org = [x for x in publisher_key if publisher[x][0] > 100000 or (publisher[x][1] and publisher[x][1] > 100000)]
print(len(famous_org))

be_select_lists = list(result_p.keys())
for fo in famous_org:
    if fo in be_select_lists:
        result.append((fo,fo,True))
    else:
        match_index = np.argmax([find_lcsubstr(fo,x) for x in be_select_lists])
        match_name = be_select_lists[match_index]
        
        match_index2 = np.argmax([longer_common_sequence(fo,x) for x in be_select_lists])
        match_name2 = be_select_lists[match_index2]
        result.append((match_name,match_name2,fo))

55


In [46]:
with open('famous_publisher.json','w') as f:
    for i in famous_publisher:
        f.write(i+'\n')

In [22]:
output = []
for k,v in result_p.items():
    if v['followers']:
        output.append({'name':k,'game_review':v['game_review'],'followers':v['followers'],'match_name':v['match_name']})

# developer

In [60]:
result_d = {}
for k,v in dict_d.items():
    if int(np.mean(v)) > 5000:
        result_d[k] = {'game_review':(int(np.mean(v)),int(np.sum(v)),len(v),np.max(v),np.max(v)/np.sum(v)),'followers':None}

In [52]:
famous_dev = [x for x in publisher_key if publisher[x][0] > 50000 or (publisher[x][1] and publisher[x][1] > 50000)]

In [104]:
result = []
be_select_lists = list(result_d.keys())
for fo in tqdm(need_search):
    if fo in be_select_lists:
        result.append((fo,fo,True))
    else:
        match_index = np.argmax([find_lcsubstr(fo,x) for x in need_search])
        match_name = be_select_lists[match_index]
        result.append((match_name,fo))

100%|██████████| 21/21 [00:00<00:00, 631.67it/s]


In [295]:
output_d = []
for k,v in result_d.items():
    output_d.append({'name':k,'game_review':v['game_review'],'followers':v['followers'],'match_name':v['match_name']})

In [297]:
output_d = sorted(output_d,key=lambda x:x['followers'][0],reverse=True)

In [108]:
famous_developer = [x[0] for x in famous_developer]

In [103]:
need_search = [x for x in wiat if x not in result_d.keys() and x not in famous_developer]

In [89]:
result_d = {}
for k,v in dict_d.items():
    result_d[k] = {'game_review':(int(np.mean(v)),int(np.sum(v)),len(v),np.max(v),np.max(v)/np.sum(v)),'followers':None}

In [96]:
result_dd = [(k,v) for k,v in result_d.items() if k not in famous_developer and v['game_review'][2] > 1]

In [97]:
result_dd = sorted(result_dd,key =lambda x:x[1]['game_review'][0],reverse=True)

In [113]:
with open('famous_developer.json','w') as f:
    for i in famous_developer:
        f.write(i+'\n')